# RNN model using library function for Sentiment Analysis

Based on the earlier preprocessing technique I have implemented a very basic RNN model in order to get an intuition and implementational details of RNN using tensorflow.

## Load Dependencies

In [1]:
import tensorflow as tf
from tqdm import tqdm, trange
import numpy as np

In [2]:
# load the saved numpy models of dataset
train = np.load('data/train_set.npz')
# reviews in form of their word indices
train_X = train['train_X']
# sentiment for reviews
train_y = train['train_y'].reshape(-1,1)
# load numpy aray containing word vectors
embed = np.load('data/embedding.npz')['embed']

The dataset is in this form 

**train_x**: (25000, seq_length)
```
[
[2, 435, 23, 34, 234, 324, 0,  0],
[1, 2,   43, 67, 23 , 20,  21, 0]
[3, 4,   2,  4,  6,   234, 45, 324],
]
```

**train_y**: (25000, 1)
```
[1, 1, 0, 0, 1]
```

**embed**: (voabulary_size, 200)
```
[
[0.06, ................. 0.04],
[0.1, 0.23 ....... 0.01, 0.06],
]
```

In [3]:
# one-hot encoding of class labels
train_y = np.concatenate([train_y, 1-train_y], axis=1)

In order to classify we use 2 nodes in output layer thus the train_y array is expanded
```
[[1,0],
 [1,0],
 [0,1],
 [1,0]]

```

In [4]:
# length of the sequences
seq_length = train_X.shape[1]
# size of each training batch
batch_size = 10
# number of iterations in training
epochs = 10
# learning rate of optimizer
learning_rate = 0.1
# shape of input layer
input_shape = 200
# shape of hidden layer
hidden_shape = 256
# shape of output layer
output_shape = 2

In [5]:
# create mini batches for training
def get_batch(x, y, batch_size, random=False, start=0, end=0):
    if random:
        # shuffled indices of batch_size
        idx = np.random.choice(range(len(x)), size=batch_size, replace=False)
    else:
        # unshuffled indices of batch_size
        idx = np.arange(start, end)
    return x[idx], y[idx]

In [6]:
tf.reset_default_graph()
with tf.Graph().as_default() as graph:
    
    # embedding tensors which contains word vectors
    embeds = tf.constant(embed, dtype=tf.float32, name="embeds")
    
    # placeholder for inputs (inputs are indices of words in sequences)
    X = tf.placeholder(shape=[None, seq_length], dtype=tf.int32, name="input")
    Y = tf.placeholder(shape=[None, output_shape], dtype=tf.float32, name="label")
    
    # placeholder for initial hidden state
    h_in = tf.placeholder(shape=[None, hidden_shape], dtype=tf.float32, name="h_in")
    
    # inputs in form of vectors (inputs are word vectors for words in sequences)
    X_embed = tf.nn.embedding_lookup(embeds, X)
    
    # the dimension of embedding vector is [batch_size, sequence_length, input_shape]
    # we need to convert it into [sequence_length, batch_size, input_shape]
    # this is done in order to perform the time unfolding of the RNN graph which is performed by tf.scan() below
    #X_embed = tf.transpose(X_embed, [1, 0, 2])
    
    
    # define RNN
    def RNN(inputs):
        with tf.variable_scope("RNN"):
            cell = tf.contrib.rnn.BasicRNNCell(hidden_shape)
            o,s = tf.nn.dynamic_rnn(cell=cell, inputs=inputs,dtype=tf.float32)
            return s
    
    
    # we only need take the hidden state output from the last time step h_out[-1]
    # this is fed into a softmax layer
    with tf.name_scope('predictions'):
        W = tf.get_variable(name="W", shape=[hidden_shape, output_shape], dtype=tf.float32)
        y = RNN(X_embed)
        preds = tf.matmul(y, W)
    
    # mean square loss function
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=preds))
        #loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(preds, Y))))
    
    # Adam optimizer for backpropagation
    with tf.name_scope('train'):
        optimize_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    with tf.Session() as sess:
        # initialize variables
        sess.run(tf.global_variables_initializer())
        
        # initialize hidden state
        h_init = np.zeros((batch_size, hidden_shape))
        
        # iterate
        for i in trange(epochs):
            
            # generate batches randomly form the train set
            batch_X, batch_y = get_batch(train_X[:100], train_y[:100], batch_size, random=True)
            
            # run the graph for optimize_op and calculate loss
            _, cost, pred = sess.run([optimize_op, loss, preds], {X:batch_X, Y:batch_y, h_in:h_init})
            
            if i%10==0:
                print(cost)
        
        writer = tf.summary.FileWriter('./tmp/1')
        writer.add_graph(sess.graph)

 10%|█         | 1/10 [00:04<00:36,  4.04s/it]

0.693147


100%|██████████| 10/10 [00:28<00:00,  2.73s/it]


In [21]:
! tensorboard --logdir ./tmp/1

Starting TensorBoard b'41' on port 6006
(You can navigate to http://127.0.1.1:6006)
^CTraceback (most recent call last):
  File "/home/shivam/anaconda3/bin/tensorboard", line 11, in <module>
    sys.exit(main())
  File "/home/shivam/anaconda3/lib/python3.5/site-packages/tensorflow/tensorboard/tensorboard.py", line 151, in main
    tb_server.serve_forever()
  File "/home/shivam/anaconda3/lib/python3.5/socketserver.py", line 232, in serve_forever
    ready = selector.select(poll_interval)
  File "/home/shivam/anaconda3/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt

